In [ ]:
import pandas as pd
import numpy as np

# ==================== CONFIGURATION ====================
# Update these paths to match your files
TOPICS_FILE = r'C:\Users\joshu\OneDrive\Desktop\CS74\Final_Project\data\weekly_topic_distributions.csv'
ECON_FILE = r'C:\Users\joshu\OneDrive\Desktop\CS74\Final_Project\music_econ_merged.csv'
OUTPUT_FILE = r'C:\Users\joshu\OneDrive\Desktop\CS74\Final_Project\music_econ_topics_merged.csv'

# ==================== LOAD DATA ====================
print("="*70)
print("LOADING DATA")
print("="*70)

topics_weekly = pd.read_csv(TOPICS_FILE)
econ_df = pd.read_csv(ECON_FILE)

print(f"Weekly topics loaded: {len(topics_weekly)} rows")
print(f"Columns: {list(topics_weekly.columns)[:5]}...")

print(f"\nEconomic data loaded: {len(econ_df)} rows")

# Convert dates
topics_weekly['week_date'] = pd.to_datetime(topics_weekly['week_date'])
econ_df['week_date'] = pd.to_datetime(econ_df['week_date'])

print(f"\nWeekly topics date range: {topics_weekly['week_date'].min()} to {topics_weekly['week_date'].max()}")
print(f"Economic data date range: {econ_df['week_date'].min()} to {econ_df['week_date'].max()}")

# Topic columns are everything except 'week_date'
all_cols = list(topics_weekly.columns)
topic_cols = [col for col in all_cols if col != 'week_date']

print(f"\nFound {len(topic_cols)} topic columns")
print(f"Topic columns: {topic_cols}")

# ==================== AGGREGATE TO MONTHLY ====================
print("\n" + "="*70)
print("AGGREGATING WEEKLY TO MONTHLY")
print("="*70)

# Extract year and month for grouping
topics_weekly['year'] = topics_weekly['week_date'].dt.year
topics_weekly['month'] = topics_weekly['week_date'].dt.month

# Average probabilities by year-month
topics_monthly = topics_weekly.groupby(['year', 'month'])[topic_cols].mean().reset_index()

# Create month-end dates to match economic data format
topics_monthly['week_date'] = pd.to_datetime(
    topics_monthly['year'].astype(str) + '-' + 
    topics_monthly['month'].astype(str).str.zfill(2) + '-01'
) + pd.offsets.MonthEnd(0)

# Drop year/month columns
topics_monthly = topics_monthly.drop(['year', 'month'], axis=1)

print(f"✓ Aggregated to {len(topics_monthly)} months")
print(f"Date range: {topics_monthly['week_date'].min()} to {topics_monthly['week_date'].max()}")

# ==================== CALCULATE DOMINANT TOPICS ====================

# Find dominant topic (highest probability)
topics_monthly['dominant_topic'] = topics_monthly[topic_cols].idxmax(axis=1)
topics_monthly['dominant_prob'] = topics_monthly[topic_cols].max(axis=1)

# Find second dominant
topics_monthly['second_topic'] = topics_monthly[topic_cols].apply(
    lambda row: row.nlargest(2).index[-1] if len(row) >= 2 else None, 
    axis=1
)
topics_monthly['second_prob'] = topics_monthly[topic_cols].apply(
    lambda row: row.nlargest(2).iloc[-1] if len(row) >= 2 else None, 
    axis=1
)

print("\nDominant topic distribution:")
print(topics_monthly['dominant_topic'].value_counts().sort_index())

# ==================== MERGE WITH ECONOMIC DATA ====================
print("\n" + "="*70)
print("MERGING WITH ECONOMIC DATA")
print("="*70)

merged_df = pd.merge(econ_df, topics_monthly, on='week_date', how='left')

print(f"\nOriginal econ data: {len(econ_df)} rows")
print(f"Monthly topic data: {len(topics_monthly)} rows")
print(f"Merged data: {len(merged_df)} rows")
print(f"Rows with topics: {merged_df['dominant_topic'].notna().sum()}")
print(f"Rows missing topics: {merged_df['dominant_topic'].isna().sum()}")

if merged_df['dominant_topic'].isna().sum() > 0:
    print("\n⚠️  Some rows missing topic data:")
    missing = merged_df[merged_df['dominant_topic'].isna()][['week_date']]
    print(f"   First missing date: {missing['week_date'].min()}")
    print(f"   Last missing date: {missing['week_date'].max()}")
    print("   (Topic data may not cover full date range)")

# ==================== ADD HUMAN-READABLE LABELS ====================

topic_labels = {
    '0': "Spiritual", '1': "Relationshp", '2': "Longing",
    '3': "Clarity", '4': "Everyday joy", '5': "Dance,Tonight's a Movie",
    '6': "Heartbreak", '7': "Intimacy", '8': "Self/Body",
    '9': "Party/Club", '10': "Bop/Dance vibes", '11': "Social Talk",
    '12': "Girl & Fun", '13': "Music & Friends", '14': "Time, Thinking, & Change",
    '15': "Appreciation", '16': "Admiring", '17': "Regret/Reflection",
    '18': "Past/Memory", '19': "Hopes & Dreams", '20': "True Love",
    '21': "Money/Power & Social Commentary", '22': "Romatinc/Intimate", '23': "Hip-Hop/Urban",
    '24': "Home", '25': "Dance & Music", '26': "Playfullness",
    '27': "Feeling Good", '28': "Freaky-Deaky", '29': "Romantic yearning"
}

merged_df['dominant_topic_label'] = merged_df['dominant_topic'].map(topic_labels)
merged_df['second_topic_label'] = merged_df['second_topic'].map(topic_labels)

# ==================== SHOW SAMPLE ====================
print("\n--- SAMPLE OF MERGED DATA ---")
sample_cols = ['week_date', 'USREC', 'acousticness', 'dominant_topic', 
               'dominant_topic_label', 'dominant_prob']

if all(col in merged_df.columns for col in sample_cols):
    print(merged_df[sample_cols].head(15).to_string(index=False))

# ==================== SAVE ====================
merged_df.to_csv(OUTPUT_FILE, index=False)
print(f"\n✓ Saved: {OUTPUT_FILE}")
print(f"Total columns: {len(merged_df.columns)}")

# ==================== ANALYZE RECESSION VS NORMAL ====================
print("\n" + "="*70)
print("TOPIC ANALYSIS: RECESSION VS NORMAL")
print("="*70)

# Filter to rows with topic data
df_with_topics = merged_df[merged_df['dominant_topic'].notna()].copy()

print(f"\nAnalyzing {len(df_with_topics)} months with topic data")

if 'USREC' in df_with_topics.columns:
    recession_count = (df_with_topics['USREC'] == 1).sum()
    normal_count = (df_with_topics['USREC'] == 0).sum()
    
    print(f"  Recession months: {recession_count}")
    print(f"  Normal months: {normal_count}")
    
    if recession_count > 0 and normal_count > 0:
        # Calculate topic distributions
        rec_topics = df_with_topics[df_with_topics['USREC'] == 1]['dominant_topic_label'].value_counts(normalize=True) * 100
        norm_topics = df_with_topics[df_with_topics['USREC'] == 0]['dominant_topic_label'].value_counts(normalize=True) * 100
        
        print("\n--- TOP 5 TOPICS DURING RECESSIONS ---")
        for topic, pct in rec_topics.head(5).items():
            print(f"  {topic:25s}: {pct:5.1f}%")
        
        print("\n--- TOP 5 TOPICS DURING NORMAL TIMES ---")
        for topic, pct in norm_topics.head(5).items():
            print(f"  {topic:25s}: {pct:5.1f}%")
        
        # Calculate differences
        all_topics = set(rec_topics.index) | set(norm_topics.index)
        differences = []
        
        for topic in all_topics:
            rec_pct = rec_topics.get(topic, 0)
            norm_pct = norm_topics.get(topic, 0)
            differences.append({
                'Topic': topic,
                'Recession %': rec_pct,
                'Normal %': norm_pct,
                'Difference': rec_pct - norm_pct
            })
        
        diff_df = pd.DataFrame(differences).sort_values('Difference', ascending=False)
        
        print("\n--- BIGGEST INCREASES DURING RECESSIONS ---")
        for _, row in diff_df.head(5).iterrows():
            print(f"  {row['Topic']:25s}: {row['Recession %']:5.1f}% vs {row['Normal %']:5.1f}% (Δ {row['Difference']:+5.1f}%)")
        
        print("\n--- BIGGEST DECREASES DURING RECESSIONS ---")
        for _, row in diff_df.tail(5).iterrows():
            print(f"  {row['Topic']:25s}: {row['Recession %']:5.1f}% vs {row['Normal %']:5.1f}% (Δ {row['Difference']:+5.1f}%)")

# ==================== SUMMARY ====================
print("\n" + "="*70)
print("READY FOR MODELING")
print("="*70)

print(f"\nYour merged dataset has {len(merged_df.columns)} columns:")
print(f"  - Original economic/audio features")
print(f"  - {len(topic_cols)} topic probability columns (0-29)")
print(f"  - 4 new dominant topic columns")

print(f"\nTo use all features in your model:")
print(f"""
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Define features
audio_features = ['danceability', 'energy', 'valence', 'tempo', 
                  'acousticness', 'instrumentalness', 'speechiness', 
                  'loudness', 'pos', 'neg']
topic_features = {topic_cols}

all_features = audio_features + topic_features

# Prepare data
df = pd.read_csv('{OUTPUT_FILE}')
df = df.dropna(subset=all_features + ['USREC'])  # Remove rows with missing data

X = df[all_features]
y = df['USREC']

# Split and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
model = LogisticRegression(class_weight='balanced', max_iter=1000)
model.fit(X_train, y_train)
""")

print("\n✓ Done!")